# Knowledge Graph Demo — Wikidata Films

This notebook demonstrates how to explore the Wikidata Films knowledge graph
stored in Neo4j. We showcase graph exploration, aggregations, and traversal
queries aligned with the project API.


In [28]:
from neo4j import GraphDatabase

NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "password"

driver = GraphDatabase.driver(
    NEO4J_URI,
    auth=(NEO4J_USER, NEO4J_PASSWORD)
)

def run_cypher(query, params=None):
    with driver.session() as session:
        return session.run(query, params or {}).data()

# Test
run_cypher("RETURN 1 AS ok")


[{'ok': 1}]

In [52]:
def explain(query, params=None):
    with driver.session() as session:
        result = session.run("EXPLAIN " + query, params or {})
        summary = result.consume()
        return summary.plan

In [53]:
def profile(query, params=None):
    with driver.session() as session:
        result = session.run("PROFILE " + query, params or {})
        summary = result.consume()
        return summary.profile

In [43]:
run_cypher("""
CALL {
  MATCH (f:Article) RETURN count(f) AS films
}
CALL {
  MATCH (d:Author) RETURN count(d) AS directors
}
CALL {
  MATCH (t:Topic) RETURN count(t) AS genres
}
RETURN films, directors, genres
""")


Received notification from DBMS server: <GqlStatusObject gql_status='01N00', status_description='warn: feature deprecated. CALL subquery without a variable scope clause is deprecated. Use CALL () { ... }', position=<SummaryInputPosition line=2, column=1, offset=1>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 1, 'line': 2, 'column': 1}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: '\nCALL {\n  MATCH (f:Article) RETURN count(f) AS films\n}\nCALL {\n  MATCH (d:Author) RETURN count(d) AS directors\n}\nCALL {\n  MATCH (t:Topic) RETURN count(t) AS genres\n}\nRETURN films, directors, genres\n'
Received notification from DBMS server: <GqlStatusObject gql_status='01N00', status_description='warn: feature deprecated. CALL subquery 

[{'films': 35, 'directors': 58, 'genres': 47}]

In [44]:
run_cypher("""
MATCH (d:Author)-[:DIRECTED]->(f:Article)
RETURN d.name AS director, count(f) AS films
ORDER BY films DESC
LIMIT 10
""")


[{'director': 'Roberto Benigni', 'films': 7},
 {'director': 'Terry Jones', 'films': 3},
 {'director': 'Éric Toledano', 'films': 1},
 {'director': 'Lina Wertmüller', 'films': 1},
 {'director': 'Matthew Vaughn', 'films': 1},
 {'director': 'Sidney Lumet', 'films': 1},
 {'director': 'Victor Fleming', 'films': 1},
 {'director': 'Sam Wood', 'films': 1},
 {'director': 'George Cukor', 'films': 1},
 {'director': 'Olivier Nakache', 'films': 1}]

In [45]:
run_cypher("""
MATCH (f:Article)-[:HAS_TOPIC]->(t:Topic)
RETURN t.name AS genre, count(f) AS films
ORDER BY films DESC
LIMIT 10
""")


[{'genre': 'drama film', 'films': 14},
 {'genre': 'comedy film', 'films': 14},
 {'genre': 'mystery film', 'films': 4},
 {'genre': 'fantasy film', 'films': 3},
 {'genre': 'science fiction film', 'films': 3},
 {'genre': 'horror film', 'films': 3},
 {'genre': 'documentary film', 'films': 3},
 {'genre': 'action film', 'films': 3},
 {'genre': 'comedy drama', 'films': 3},
 {'genre': 'biographical film', 'films': 2}]

In [56]:
run_cypher("""
MATCH (t:Topic)
RETURN DISTINCT t.name AS genre
ORDER BY genre
""")

[{'genre': 'LGBT-related film'},
 {'genre': 'action film'},
 {'genre': 'adventure film'},
 {'genre': 'biographical film'},
 {'genre': "children's film"},
 {'genre': 'cinematic fairy tale'},
 {'genre': 'comedy'},
 {'genre': 'comedy drama'},
 {'genre': 'comedy film'},
 {'genre': 'coming-of-age film'},
 {'genre': 'crime film'},
 {'genre': 'documentary film'},
 {'genre': 'drama fiction'},
 {'genre': 'drama film'},
 {'genre': 'epic film'},
 {'genre': 'family film'},
 {'genre': 'fantasy film'},
 {'genre': 'fiction film'},
 {'genre': 'film based on a novel'},
 {'genre': 'film noir'},
 {'genre': 'flashback film'},
 {'genre': 'ghost film'},
 {'genre': 'heist film'},
 {'genre': 'horror film'},
 {'genre': 'huis-clos film'},
 {'genre': 'kung fu film'},
 {'genre': 'live-action/animated film'},
 {'genre': 'melodrama'},
 {'genre': 'military science fiction'},
 {'genre': 'musical film'},
 {'genre': 'mystery film'},
 {'genre': 'neo-noir'},
 {'genre': 'parody film'},
 {'genre': 'period drama film'},
 {'

In [55]:
profile("""
MATCH (t:Topic {name: "Drama"})<-[:HAS_TOPIC]-(f:Article)
OPTIONAL MATCH (d:Author)-[:DIRECTED]->(f)
RETURN f.title AS film, d.name AS director, f.year AS year
LIMIT 15
""")

{'args': {'GlobalMemory': 64,
  'planner-impl': 'IDP',
  'Memory': 0,
  'string-representation': 'Cypher 5\n\nPlanner COST\n\nRuntime SLOTTED\n\nRuntime version 5.26\n\n+----------------------+----+-------------------------------------------------------------------+----------------+------+---------+----------------+------------------------+\n| Operator             | Id | Details                                                           | Estimated Rows | Rows | DB Hits | Memory (Bytes) | Page Cache Hits/Misses |\n+----------------------+----+-------------------------------------------------------------------+----------------+------+---------+----------------+------------------------+\n| +ProduceResults      |  0 | film, director, year                                              |              4 |    0 |       0 |              0 |                    0/0 |\n| |                    +----+-------------------------------------------------------------------+----------------+------+---------+

In [49]:
run_cypher("""
MATCH (a1:Author), (a2:Author)
WHERE a1 <> a2
MATCH p = shortestPath((a1)-[*1..6]-(a2))
RETURN a1.name, a2.name, length(p) AS hops
LIMIT 5

""")


[{'a1.name': 'Ondi Timoner', 'a2.name': 'Todd McCarthy', 'hops': 4},
 {'a1.name': 'Ondi Timoner', 'a2.name': 'Ettore Scola', 'hops': 4},
 {'a1.name': 'Ondi Timoner', 'a2.name': 'Francesco Maselli', 'hops': 4},
 {'a1.name': 'Ondi Timoner', 'a2.name': 'Luigi Magni', 'hops': 4},
 {'a1.name': 'Ondi Timoner', 'a2.name': 'Ansano Giannarelli', 'hops': 4}]

In [54]:
explain("""
MATCH (f:Article)
WHERE f.title CONTAINS "Love"
RETURN f
LIMIT 10
""")


{'args': {'planner-impl': 'IDP',
  'Details': 'f',
  'planner-version': '5.26',
  'string-representation': 'Cypher 5\n\nPlanner COST\n\nRuntime SLOTTED\n\nRuntime version 5.26\n\n+-----------------+----+----------------------------------------------------------------------+----------------+\n| Operator        | Id | Details                                                              | Estimated Rows |\n+-----------------+----+----------------------------------------------------------------------+----------------+\n| +ProduceResults |  0 | f                                                                    |              0 |\n| |               +----+----------------------------------------------------------------------+----------------+\n| +Limit          |  1 | 10                                                                   |              0 |\n| |               +----+----------------------------------------------------------------------+----------------+\n| +Filter         |  2 

## Conclusion

This notebook demonstrates how Neo4j enables efficient exploration of
a real-world film dataset using graph traversals, aggregations,
and pathfinding queries. These queries complement the REST API
implemented in FastAPI.
